In [37]:
import numpy as np
import bm3d
import matplotlib.pyplot as plt
from pathlib import Path
from skimage import io, img_as_float
import os
import glob
from collections import defaultdict

File Paths

In [38]:
# Path to the input dataset
icip_data = "D:/VIP Cup/datasets/ICIP training data/"

# Path to the denoised output dataset
#final_denoised_dataset = "D:/VIP Cup/output_datasets/denoised_icip_dataset"
final_denoised_dataset = "D:/VIP Cup/temp_data"

Constants

In [39]:
tiff_directories = defaultdict(int)
denoised_tiff_count = 0
original_tiff_count = 0
SIGMA = 0.085
oct_height = 300
oct_width = 300

Functions to check whether a tiff file/directory or not

In [40]:
def is_tiff_dir(curr_path):
    '''function to check if the current directory having any tiff files'''
    num_tiff = len(glob.glob(os.path.join(curr_path, '*.tiff' or '*.tif')))
    return num_tiff

def is_tiff(curr_path):
    '''function to check if the current file is a tiff file or not'''
    if os.path.isfile(curr_path) and ((".tiff" or ".tif" ) in curr_path):
        return True
    else:
        return False

Searching tiff directories

In [41]:
tiff_directories = defaultdict(int)
denoised_tiff_count = 0
original_tiff_count = 0
SIGMA = 0.083

dataset = glob.iglob(os.path.join(icip_data, '**'), recursive=True)

for i,file in enumerate(dataset):
    if is_tiff_dir(file):

        current_tiff_count = is_tiff_dir(file)
        tiff_directories[file] = current_tiff_count
        print(f"{len(tiff_directories)}th case {file} ......")
        original_tiff_count += current_tiff_count



print(f"Denoised cases: {len(tiff_directories)}")
#print(f"Denoised tiff images: {denoised_tiff_count}")
print(f"Original tiff images: {original_tiff_count}")


1th case D:/VIP Cup/datasets/ICIP training data\0\RawDataQA (1) ......
2th case D:/VIP Cup/datasets/ICIP training data\0\RawDataQA (10) ......
3th case D:/VIP Cup/datasets/ICIP training data\0\RawDataQA (11) ......
4th case D:/VIP Cup/datasets/ICIP training data\0\RawDataQA (12) ......
5th case D:/VIP Cup/datasets/ICIP training data\0\RawDataQA (13) ......
6th case D:/VIP Cup/datasets/ICIP training data\0\RawDataQA (14) ......
7th case D:/VIP Cup/datasets/ICIP training data\0\RawDataQA (15) ......
8th case D:/VIP Cup/datasets/ICIP training data\0\RawDataQA (16) ......
9th case D:/VIP Cup/datasets/ICIP training data\0\RawDataQA (17) ......
10th case D:/VIP Cup/datasets/ICIP training data\0\RawDataQA (18) ......
11th case D:/VIP Cup/datasets/ICIP training data\0\RawDataQA (19) ......
12th case D:/VIP Cup/datasets/ICIP training data\0\RawDataQA (2) ......
13th case D:/VIP Cup/datasets/ICIP training data\0\RawDataQA (20) ......
14th case D:/VIP Cup/datasets/ICIP training data\0\RawDataQA (

Denoising Function

In [42]:
import numpy as np
from numpy.linalg import svd
from PIL import Image
import math

def get_svd_image_from_array(A):
    # Convert the image to a NumPy array

    # Remember image size
    n = len(A)

    # Take SVD
    U, S, V = svd(A)

    threshold = 4/np.sqrt(3)
    filter = (S > threshold)*1 
    filtered_S = np.multiply(S, filter)

    # Initialize blank image
    final_img = np.zeros((n, n))
    
    for i in range(n):
        matrix = S[i]*np.outer(U[:,i],V[i])
        final_img += matrix
    
    return final_img

In [43]:
def open_noisy_image_as_array(image_path):
    image_array = img_as_float(io.imread(image_path, as_gray=True))
    return np.array(image_array)

In [44]:
def bm3d_denoise(noisy_image, npsd_sigma = SIGMA):

    noisy_oct = img_as_float(io.imread(noisy_image, as_gray=True))
    sigma = npsd_sigma
    bm3d_denoised = bm3d.bm3d(noisy_oct, sigma_psd=sigma, stage_arg=bm3d.BM3DStages.ALL_STAGES)
    return bm3d_denoised

Saving Denoised Image as a .tiff file

In [45]:
def save_denoised_image_as_tiff(denoised_array, output_path):
    
    plt.imsave(output_path, denoised_array, cmap='gray')

Output path generating function

In [46]:
def generate_out_path(file, output_dir, sigma):
    '''generate output path'''
    file_name = os.path.basename(file).split(".")[0]
    case_name = os.path.basename(os.path.dirname(file))
    class_name = os.path.basename(os.path.dirname(os.path.dirname(file)))

    case_dir = os.path.join(output_dir,class_name,case_name)
    os.makedirs(case_dir, exist_ok=True)

    out_path = os.path.join(case_dir, file_name + "_denoised_.tiff")
    return out_path

Denoising Function

In [47]:
#denoising function
def denoise(image_path, output_path, sigma=SIGMA):
    '''function to denoise the image'''
    bm3d_denoised = bm3d_denoise(image_path, sigma)
    svd_denoised = get_svd_image_from_array(bm3d_denoised)
    save_denoised_image_as_tiff(svd_denoised, output_path)

In [48]:
tiff_directories

defaultdict(int,
            {'D:/VIP Cup/datasets/ICIP training data\\0\\RawDataQA (1)': 52,
             'D:/VIP Cup/datasets/ICIP training data\\0\\RawDataQA (10)': 273,
             'D:/VIP Cup/datasets/ICIP training data\\0\\RawDataQA (11)': 300,
             'D:/VIP Cup/datasets/ICIP training data\\0\\RawDataQA (12)': 258,
             'D:/VIP Cup/datasets/ICIP training data\\0\\RawDataQA (13)': 146,
             'D:/VIP Cup/datasets/ICIP training data\\0\\RawDataQA (14)': 220,
             'D:/VIP Cup/datasets/ICIP training data\\0\\RawDataQA (15)': 87,
             'D:/VIP Cup/datasets/ICIP training data\\0\\RawDataQA (16)': 52,
             'D:/VIP Cup/datasets/ICIP training data\\0\\RawDataQA (17)': 176,
             'D:/VIP Cup/datasets/ICIP training data\\0\\RawDataQA (18)': 300,
             'D:/VIP Cup/datasets/ICIP training data\\0\\RawDataQA (19)': 84,
             'D:/VIP Cup/datasets/ICIP training data\\0\\RawDataQA (2)': 201,
             'D:/VIP Cup/datasets/ICIP tr

Denoising Pipeline

In [49]:
# denoising pipeline for denoise the tiff images in each and every tiff directories
def denoising_pipeline(tiff_directories, final_denoised_dataset, SIGMA):
    '''denoising pipeline for denoise the tiff images in each and every tiff directories'''
    count = 0

    for i, tiff_dir in enumerate(tiff_directories):
        print(f"{i}th case {tiff_dir} ......")

        tiff_files = os.listdir(tiff_dir)

        if len(tiff_files) == tiff_directories[tiff_dir]:
            for tiff_file in tiff_files:
                tiff_file_path = os.path.join(tiff_dir, tiff_file)
                output_path = generate_out_path(tiff_file_path, final_denoised_dataset, SIGMA)
                denoise(tiff_file_path, output_path, SIGMA)
        else:
            raise Exception(f"Number of tiff files in the directory is not equal to the number of tiff files in the dictionary in {tiff_dir}")

        print(f"{i}th case {tiff_dir} denoised ......")
        count += 1

    return count
count = 0

In [ ]:
# denoising pipeline for denoise the tiff images in each and every tiff directories
denoised_tiff_count = denoising_pipeline(tiff_directories, final_denoised_dataset, SIGMA)
print(f"Denoised cases: {denoised_tiff_count}")